# COMP 4462 Data Visualization Tutorial 5: Pandas and Visualization

In this tutorial, we will learn more about Pandas and Data Visualization with Python.

You can view this notebook on Google Colab ([https://bit.ly/vis-t05-nb](https://bit.ly/vis-t05-nb)).

All the materials of this tutorial are hosted on [this GitHub repository](https://github.com/leoyuholo/learning-vis-tools/tree/master/tutorial05).

## Dataset

### Download dataset

We will explore the [Spotify"s Worldwide Daily Song Ranking](https://www.kaggle.com/edumucelli/spotifys-worldwide-daily-song-ranking) prepared by Eduardo on Kaggle.

Since the dataset is not tiny enough for the whole class to upload simultaneously, it is shared on Google Drive (https://drive.google.com/open?id=1ki4HKtNKpj-Ewj5RGToXpMrzgXh4ZIbz) and we will download it directly into the Google Colab workspace. The zipped file is about 43MB and the unzipped file contains more than 3 million records and size over 350MB.

Any commands that starts with `!` are shell commands, they are not Python code. Instead, you can run commands like `ls`, `mkdir`, etc. in Jupyter notebook. In the following case, we use `pip`, a command to install Python package, then use the installed package `gdown` to download the file shared on Google Drive. Finally, we unzip the file to obtain the dataset we will use throughout this tutorial.

In [1]:
# Download dataset, it will take about 1 min

!pip -q install gdown
!gdown https://drive.google.com/uc?id=1ki4HKtNKpj-Ewj5RGToXpMrzgXh4ZIbz
!unzip spotifys-worldwide-daily-song-ranking.zip

Downloading...
From: https://drive.google.com/uc?id=1ki4HKtNKpj-Ewj5RGToXpMrzgXh4ZIbz
To: /content/spotifys-worldwide-daily-song-ranking.zip
45.2MB [00:00, 144MB/s] 
Archive:  spotifys-worldwide-daily-song-ranking.zip
  inflating: data.csv                


## Explore the dataset with Pandas

In [0]:
# Import pandas

import pandas as pd

In [0]:
# Since the dataset is about 350MB and contains over 3 million records, it will 
# take some time to load the dataset

df_daily_ranking = pd.read_csv("data.csv")

In [4]:
# Show the first 5 rows of the data

df_daily_ranking.head()

,Position,Track Name,Artist,Streams,URL,Date,Region
0,1,Reggaetón Lento (Bailemos),CNCO,19272,https://open.spotify.com/track/3AEZUABDXNtecAO...,2017-01-01,ec
1,2,Chantaje,Shakira,19270,https://open.spotify.com/track/6mICuAdrwEjh6Y6...,2017-01-01,ec
2,3,Otra Vez (feat. J Balvin),Zion & Lennox,15761,https://open.spotify.com/track/3QwBODjSEzelZyV...,2017-01-01,ec
3,4,Vente Pa' Ca,Ricky Martin,14954,https://open.spotify.com/track/7DM4BPaS7uofFul...,2017-01-01,ec
4,5,Safari,J Balvin,14269,https://open.spotify.com/track/6rQSrBHf7HlZjtc...,2017-01-01,ec


In [5]:
# Compute the statistical summary of all the columns

df_daily_ranking.describe(include="all")

,Position,Track Name,Artist,Streams,URL,Date,Region
count,3.441197e+06,3440540,3440540,3.441197e+06,3441189,3441197,3441197
unique,NaN,18597,6628,NaN,21746,371,54
top,NaN,Shape of You,Ed Sheeran,NaN,https://open.spotify.com/track/7qiZfU4dY1lWllz...,2017-12-24,cr
freq,NaN,19365,127064,NaN,19365,10073,74200
mean,9.464399e+01,NaN,NaN,5.189176e+04,NaN,NaN,NaN
std,5.739567e+01,NaN,NaN,2.018035e+05,NaN,NaN,NaN
min,1.000000e+00,NaN,NaN,1.001000e+03,NaN,NaN,NaN
25%,4.500000e+01,NaN,NaN,3.322000e+03,NaN,NaN,NaN
50%,9.200000e+01,NaN,NaN,9.227000e+03,NaN,NaN,NaN
75%,1.430000e+02,NaN,NaN,2.965800e+04,NaN,NaN,NaN


## Clean up

### See if any rows contain null

In [6]:
# Show 5 rows that contain null in any columns
# We will revisit the this selector syntax later

df_daily_ranking[df_daily_ranking.isnull().any(axis=1)].head()

,Position,Track Name,Artist,Streams,URL,Date,Region
39428,29,NaN,NaN,7362,https://open.spotify.com/track/3RXkboS74UYzN14...,2017-07-20,ec
39456,57,NaN,NaN,4426,https://open.spotify.com/track/4JAyIDXOqNM6qHu...,2017-07-20,ec
39463,64,NaN,NaN,4069,https://open.spotify.com/track/3bVbQvGVIe4n24A...,2017-07-20,ec
39490,91,NaN,NaN,3238,https://open.spotify.com/track/3eFJqPe8VUYrABb...,2017-07-20,ec
39631,32,NaN,NaN,7409,https://open.spotify.com/track/3RXkboS74UYzN14...,2017-07-21,ec


In [7]:
# Counting how many records have null values, for each column

df_daily_ranking.isnull().sum()

Position        0
Track Name    657
Artist        657
Streams         0
URL             8
Date            0
Region          0
dtype: int64

### Drop the null rows

In [0]:
# Drop all the rows that contain any null values
# Inplace avoid copying the whole dataframe, computationally faster

df_daily_ranking.dropna(inplace=True)

### Separate "global" from the rest

In the dataset, there is a region "global", which is the sum of all the streams all over the world. It overlaps with other regions, treating it as equally as a region will cause "double count", so we separate it out.

In [0]:
df_daily_ranking_global = df_daily_ranking[df_daily_ranking['Region'] == 'global']
df_daily_ranking = df_daily_ranking[df_daily_ranking['Region'] != 'global']

## Aggregate

### Top artists of the year

In [10]:
# Count how many unique artists in the dataset

df_daily_ranking["Artist"].nunique()

6628

The following command has multiple components, `groupby`, `sum`, `sort_values`, column selection and `head`, we look into it one by one:

- `groupby`: This is like splitting the whole dataset by the "Artist" of each row, resulting in 6628 groups.
- `sum`: As there are multiple rows in each group of 6628 groups, this command sums up the values of each group. In our case, it sums up the "Streams" column of each row in each group.
- `sort_values`: After summing up, each group becomes one row of record, that is 6628 rows now. This command sort these 6628 rows by their value in "Streams" in descending order.
- select the "Streams" column: This is like ignoring other columns and only show us the "Streams" column alongside with the index column (which is the column used in the `groupby` command).
- `head`: Show the first 5 rows

In [11]:
# Show the top 5 artists of the year

df_daily_ranking.groupby("Artist").sum().sort_values("Streams", ascending=False)["Streams"].head()

Artist
Ed Sheeran          4353885528
Drake               2285102445
The Chainsmokers    2081716050
Post Malone         1865412162
Luis Fonsi          1760377876
Name: Streams, dtype: int64

### Top songs of the year

In [0]:
################################################################################
# TODO:                                                                        #
# Try to show the top 5 songs of the year!                                     #
################################################################################



################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

## Filter

### Top song of the year in US

The square brackets of pandas have multiple usage, it is sometimes even confusing. For now, we look into using it as column selection and filtering.

If the value inside the brackets is a string, it will select the column, just like `df_daily_ranking["Region"]` will return a column.

If the value inside the brackets is an array of boolean values, it will return only the `True` rows.

Let"s see an example!

In [13]:
# Ignore the code below at the moment, all we care is "df_sample" now contains 
# 10 rows, and 5 are in Region "us" and 5 in Region "hk"

df_sample = pd.concat([df_daily_ranking[df_daily_ranking["Region"] == "us"].head(), df_daily_ranking[df_daily_ranking["Region"] == "hk"].head()])
df_sample

,Position,Track Name,Artist,Streams,URL,Date,Region
771622,1,Bad and Boujee (feat. Lil Uzi Vert),Migos,1371493,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,2017-01-01,us
771623,2,Fake Love,Drake,1180074,https://open.spotify.com/track/343YBumqHu19cGo...,2017-01-01,us
771624,3,Starboy,The Weeknd,1064351,https://open.spotify.com/track/5aAx2yezTd8zXrk...,2017-01-01,us
771625,4,Closer,The Chainsmokers,1010492,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,2017-01-01,us
771626,5,Black Beatles,Rae Sremmurd,874289,https://open.spotify.com/track/6fujklziTHa8uoM...,2017-01-01,us
3366997,1,Closer,The Chainsmokers,12831,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,2017-01-01,hk
3366998,2,Starboy,The Weeknd,9626,https://open.spotify.com/track/5aAx2yezTd8zXrk...,2017-01-01,hk
3366999,3,Let Me Love You,DJ Snake,9499,https://open.spotify.com/track/4pdPtRcBmOSQDlJ...,2017-01-01,hk
3367000,4,I Don’t Wanna Live Forever (Fifty Shades Darke...,ZAYN,8210,https://open.spotify.com/track/3NdDpSvN911VPGi...,2017-01-01,hk
3367001,5,Don't Wanna Know,Maroon 5,7606,https://open.spotify.com/track/5MFzQMkrl1FOOng...,2017-01-01,hk


In [14]:
# To see how filter works, we now see how the "==" operator works

df_sample["Region"] == "us"

771622      True
771623      True
771624      True
771625      True
771626      True
3366997    False
3366998    False
3366999    False
3367000    False
3367001    False
Name: Region, dtype: bool

While the left column is the row id, the right column is the truth value of whether the row has "Region" equals "us". By passing this array into the dataframe, it will filter out only the rows marked as `True`, that is the first 5 rows.

In [15]:
# Only keep rows with "Region" equals "us"

df_sample[df_sample["Region"] == "us"]

,Position,Track Name,Artist,Streams,URL,Date,Region
771622,1,Bad and Boujee (feat. Lil Uzi Vert),Migos,1371493,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,2017-01-01,us
771623,2,Fake Love,Drake,1180074,https://open.spotify.com/track/343YBumqHu19cGo...,2017-01-01,us
771624,3,Starboy,The Weeknd,1064351,https://open.spotify.com/track/5aAx2yezTd8zXrk...,2017-01-01,us
771625,4,Closer,The Chainsmokers,1010492,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,2017-01-01,us
771626,5,Black Beatles,Rae Sremmurd,874289,https://open.spotify.com/track/6fujklziTHa8uoM...,2017-01-01,us


In [16]:
# By applying the same logic, we filter the whole dataset

df_daily_ranking_in_us = df_daily_ranking[df_daily_ranking["Region"] == "us"]

# Then, do the aggregation and find out the top songs in US

df_daily_ranking_in_us.groupby("Artist").sum().sort_values("Streams", ascending=False)["Streams"].head()

Artist
Drake             1250864578
Kendrick Lamar    1163890899
Post Malone        988811897
Lil Uzi Vert       773675118
Ed Sheeran         723507889
Name: Streams, dtype: int64

### Top song of the year in Hong Kong

In [0]:
################################################################################
# TODO:                                                                        #
# Try to show the top 5 songs of the year in Hong Kong!                        #
################################################################################



################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

## Read dataset in JSON format

In order to find out who are the top artists or the top songs in Asia / North America / Europe, we need to link up the country code to continents. To do so, we need an extra dataset. You can download the `country.json` from the [tutorial material repository on GitHub](https://github.com/leoyuholo/learning-vis-tools/tree/master/tutorial04/lab3). Thanks to the authors of the GitHub repository [annexare/Countries](https://github.com/annexare/Countries).

If you are using Google Colab, you should be able to see an arrow on the left edge of the window, it hides the panel. Open it and switch to the "Files" tab, you can see the files in the current directory. Click upload and upload the dataset you have downloaded.

In [18]:
# Read JSON file into dataframe

df_countries = pd.read_json("countries.json")
df_countries

,AD,AE,AF,AG,AI,AL,AM,AO,AQ,AR,...,VN,VU,WF,WS,XK,YE,YT,ZA,ZM,ZW
capital,Andorra la Vella,Abu Dhabi,Kabul,Saint John's,The Valley,Tirana,Yerevan,Luanda,,Buenos Aires,...,Hanoi,Port Vila,Mata-Utu,Apia,Pristina,Sana'a,Mamoudzou,Pretoria,Lusaka,Harare
continent,EU,AS,AS,NA,NA,EU,AS,AF,AN,SA,...,AS,OC,OC,OC,EU,AS,AF,AF,AF,AF
currency,EUR,AED,AFN,XCD,XCD,ALL,AMD,AOA,,ARS,...,VND,VUV,XPF,WST,EUR,YER,EUR,ZAR,ZMK,ZWL
languages,[ca],[ar],"[ps, uz, tk]",[en],[en],[sq],"[hy, ru]",[pt],[],"[es, gn]",...,[vi],"[bi, en, fr]",[fr],"[sm, en]","[sq, sr]",[ar],[fr],"[af, en, nr, st, ss, tn, ts, ve, xh, zu]",[en],"[en, sn, nd]"
name,Andorra,United Arab Emirates,Afghanistan,Antigua and Barbuda,Anguilla,Albania,Armenia,Angola,Antarctica,Argentina,...,Vietnam,Vanuatu,Wallis and Futuna,Samoa,Kosovo,Yemen,Mayotte,South Africa,Zambia,Zimbabwe
native,Andorra,دولة الإمارات العربية المتحدة,افغانستان,Antigua and Barbuda,Anguilla,Shqipëria,Հայաստան,Angola,Antarctica,Argentina,...,Việt Nam,Vanuatu,Wallis et Futuna,Samoa,Republika e Kosovës,اليَمَن,Mayotte,South Africa,Zambia,Zimbabwe
phone,376,971,93,1268,1264,355,374,244,672,54,...,84,678,681,685,"377,381,383,386",967,262,27,260,263


In [19]:
# Flip the rows and columns

df_countries.transpose().head()

,capital,continent,currency,languages,name,native,phone
AD,Andorra la Vella,EU,EUR,[ca],Andorra,Andorra,376
AE,Abu Dhabi,AS,AED,[ar],United Arab Emirates,دولة الإمارات العربية المتحدة,971
AF,Kabul,AS,AFN,"[ps, uz, tk]",Afghanistan,افغانستان,93
AG,Saint John's,NA,XCD,[en],Antigua and Barbuda,Antigua and Barbuda,1268
AI,The Valley,NA,XCD,[en],Anguilla,Anguilla,1264


In [0]:
# The argument "orient" does the exactly same thing

df_countries = pd.read_json("countries.json", orient="index")

## Join

In [0]:
# Since the country code is in lower case, to match up, we need to make them 
# upper case

df_daily_ranking["country"] = df_daily_ranking["Region"].str.upper()

In [0]:
# Merge the two dataframe on "country" column of df_daily_ranking and on the 
# index column of df_country

df_daily_ranking_with_continent = df_daily_ranking.merge(df_countries, how="inner", left_on="country", right_index=True)

In [0]:
# Rename column, just as an example, no specific purpose

df_daily_ranking_with_continent = df_daily_ranking_with_continent.rename(columns={"continent": "Continent"})

### Top artists of the year in North America

In [24]:
# Agggregate just the same as region

df_daily_ranking_in_na = df_daily_ranking_with_continent[df_daily_ranking_with_continent["Continent"] == "NA"]
df_daily_ranking_in_na.groupby("Artist").sum().sort_values("Streams", ascending=False)["Streams"].head(20)

Artist
Drake               1488701744
Kendrick Lamar      1281915448
Post Malone         1125828758
Ed Sheeran          1083913228
Lil Uzi Vert         837881663
Migos                764677867
The Chainsmokers     762650964
Future               627364617
The Weeknd           599949693
Luis Fonsi           582328513
21 Savage            524445998
Khalid               503672138
Kodak Black          482252053
Travis Scott         471269591
J Balvin             465769112
Calvin Harris        459310805
DJ Khaled            456748400
XXXTENTACION         439677187
Imagine Dragons      430717997
Big Sean             428376262
Name: Streams, dtype: int64

### Top song of the year in Asia

In [0]:
################################################################################
# TODO:                                                                        #
# Try to show the top 5 songs of the year in Asia!                             #
################################################################################



################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

## Pivot Table

In [26]:
# Filter the artists

df_favorite_artist = df_daily_ranking_with_continent[df_daily_ranking_with_continent["Artist"] == "Coldplay"]
df_favorite_artist.head()

,Position,Track Name,Artist,Streams,URL,Date,Region,country,capital,Continent,currency,languages,name,native,phone
83,84,Hymn For The Weekend - Seeb Remix,Coldplay,2327,https://open.spotify.com/track/1OAiWI2oPmglaOi...,2017-01-01,ec,EC,Quito,SA,USD,[es],Ecuador,Ecuador,593
185,186,The Scientist,Coldplay,1285,https://open.spotify.com/track/75JFxkI2RXiU7L9...,2017-01-01,ec,EC,Quito,SA,USD,[es],Ecuador,Ecuador,593
195,196,Adventure Of A Lifetime,Coldplay,1236,https://open.spotify.com/track/69uxyAqqPIsUyTO...,2017-01-01,ec,EC,Quito,SA,USD,[es],Ecuador,Ecuador,593
268,69,Hymn For The Weekend - Seeb Remix,Coldplay,2519,https://open.spotify.com/track/1OAiWI2oPmglaOi...,2017-01-02,ec,EC,Quito,SA,USD,[es],Ecuador,Ecuador,593
301,102,The Scientist,Coldplay,1998,https://open.spotify.com/track/75JFxkI2RXiU7L9...,2017-01-02,ec,EC,Quito,SA,USD,[es],Ecuador,Ecuador,593


In [27]:
# Pivot table, just like spreadsheet, define which attribute goes to rows, which
# to columns, values, and last but not least, how to aggregate them. Aggregation
# can be sum, max, min, mean, etc. In our case, we want to add up the streams 
# counts, so we use "sum"

df_favorite_artist_streams = df_favorite_artist.pivot_table(values="Streams", index="Date", columns="Continent", aggfunc="sum")
df_favorite_artist_streams.head()

Continent,AS,EU,NA,OC,SA
Date,,,,,
2017-01-01,134203.0,427319.0,274971.0,26141.0,215578.0
2017-01-02,155686.0,512328.0,318189.0,28136.0,293633.0
2017-01-03,174060.0,525427.0,370151.0,28885.0,318359.0
2017-01-04,197094.0,531836.0,353325.0,32528.0,324242.0
2017-01-05,198938.0,504666.0,345913.0,28291.0,320139.0


In [0]:
################################################################################
# TODO:                                                                        #
# Try to make a pivot table for your favorite song!                            #
################################################################################



################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

## Altair

In [0]:
import altair as alt

In [0]:
# In case you hit the error message of 5000 rows limitation, you can run the 
# following command to disable the limitation

# alt.data_transformers.enable('default', max_rows=None)

In [31]:
# Same "groupby" trick we used before. The attributes used for "groupby" will 
# becomes index of the dataframe, which are no longer normal columns and cannot 
# be used directly. The method "reset_index" pulls out the attributes used for 
# "groupby" back to normal columns

df_chart = df_favorite_artist.groupby(['Date', 'Continent']).sum()['Streams'].reset_index()
df_chart.head(10)

,Date,Continent,Streams
0,2017-01-01,AS,134203
1,2017-01-01,EU,427319
2,2017-01-01,NA,274971
3,2017-01-01,OC,26141
4,2017-01-01,SA,215578
5,2017-01-02,AS,155686
6,2017-01-02,EU,512328
7,2017-01-02,NA,318189
8,2017-01-02,OC,28136
9,2017-01-02,SA,293633


Plot dataframe as line chart. The altair API is designed in visualization language. We can apply the knowledge learned in lectures, what we want to encode with each encoding channels. See the documentation for [marks](https://altair-viz.github.io/user_guide/marks.html) and [encodings](https://altair-viz.github.io/user_guide/encoding.html).

In [32]:
# State what we want the X-axis to encode, and what for the Y-axis, finally, 
# what for the color channel
# Data types ("Q": quantitative, "O": ordinal, "N": nominal, "T": 
# temporal) are stated with the delimiter ":" after the column name
# See documentation for more details

alt.Chart(df_chart).mark_line().encode(
    x="Date:T",
    y="Streams:Q",
    color="Continent:N"
)

Chart({
  data:             Date Continent  Streams
  0     2017-01-01        AS   134203
  1     2017-01-01        EU   427319
  2     2017-01-01        NA   274971
  3     2017-01-01        OC    26141
  4     2017-01-01        SA   215578
  5     2017-01-02        AS   155686
  6     2017-01-02        EU   512328
  7     2017-01-02        NA   318189
  8     2017-01-02        OC    28136
  9     2017-01-02        SA   293633
  10    2017-01-03        AS   174060
  11    2017-01-03        EU   525427
  12    2017-01-03        NA   370151
  13    2017-01-03        OC    28885
  14    2017-01-03        SA   318359
  15    2017-01-04        AS   197094
  16    2017-01-04        EU   531836
  17    2017-01-04        NA   353325
  18    2017-01-04        OC    32528
  19    2017-01-04        SA   324242
  20    2017-01-05        AS   198938
  21    2017-01-05        EU   504666
  22    2017-01-05        NA   345913
  23    2017-01-05        OC    28291
  24    2017-01-05        SA   320139
  25    2017-01-06        AS   172680
  26    2017-01-06        EU   433477
  27    2017-01-06        NA   191062
  28    2017-01-06        OC    27989
  29    2017-01-06        SA   317580
  ...          ...       ...      ...
  1520  2018-01-02        NA    45909
  1521  2018-01-02        SA    46169
  1522  2018-01-03        AS    91611
  1523  2018-01-03        EU    42272
  1524  2018-01-03        NA    50616
  1525  2018-01-03        SA    33299
  1526  2018-01-04        AS    91506
  1527  2018-01-04        EU    55970
  1528  2018-01-04        NA    52406
  1529  2018-01-04        SA    33952
  1530  2018-01-05        AS    91929
  1531  2018-01-05        EU    34485
  1532  2018-01-05        NA    53691
  1533  2018-01-05        SA    33379
  1534  2018-01-06        AS    78961
  1535  2018-01-06        EU    33853
  1536  2018-01-06        NA    48913
  1537  2018-01-06        SA    27371
  1538  2018-01-07        AS    81563
  1539  2018-01-07        EU    35331
  1540  2018-01-07        NA    42202
  1541  2018-01-07        SA    25648
  1542  2018-01-08        AS    93803
  1543  2018-01-08        EU    44195
  1544  2018-01-08        NA    52670
  1545  2018-01-08        SA    47043
  1546  2018-01-09        AS   103601
  1547  2018-01-09        EU    36796
  1548  2018-01-09        NA    54762
  1549  2018-01-09        SA    34694
  
  [1550 rows x 3 columns],
  encoding: EncodingWithFacet({
    color: Color({
      shorthand: 'Continent:N'
    }),
    x: X({
      shorthand: 'Date:T'
    }),
    y: Y({
      shorthand: 'Streams:Q'
    })
  }),
  mark: 'line'
})

### Juxtaposition (side-by-side)

Altair makes it very easy to put multiple charts together, see the [documentation](https://altair-viz.github.io/user_guide/compound_charts.html) for more examples and how to use this powerful feature.

In [33]:
# Filter out the song we want to visualize

df_chart = df_daily_ranking_global[df_daily_ranking_global["Track Name"] == "Something Just Like This"]
df_chart.head()

,Position,Track Name,Artist,Streams,URL,Date,Region
3127244,2,Something Just Like This,The Chainsmokers,4752225,https://open.spotify.com/track/6RUKPb4LETWmmr3...,2017-02-24,global
3127444,2,Something Just Like This,The Chainsmokers,4460815,https://open.spotify.com/track/6RUKPb4LETWmmr3...,2017-02-25,global
3127644,2,Something Just Like This,The Chainsmokers,3949869,https://open.spotify.com/track/6RUKPb4LETWmmr3...,2017-02-26,global
3127844,2,Something Just Like This,The Chainsmokers,4374275,https://open.spotify.com/track/6RUKPb4LETWmmr3...,2017-02-27,global
3128044,2,Something Just Like This,The Chainsmokers,4517368,https://open.spotify.com/track/6RUKPb4LETWmmr3...,2017-02-28,global


In [34]:
# We store the return values in variables, and then we can use "|", "&" and "+" 
# to combine them as horizontally, vertically or overlay, see documentation for 
# more details

position_chart = alt.Chart(df_chart).mark_line().encode(
    x='Date:T',
    y=alt.Y('Position:Q', scale=alt.Scale(zero=False, padding=1, domain=[200, 1]))
)

streams_chart = alt.Chart(df_chart).mark_bar().encode(
    x='Date:T',
    y='Streams:Q'
)

position_chart | streams_chart

HConcatChart({
  hconcat: [Chart({
    data:          Position                Track Name            Artist  Streams  \
    3127244         2  Something Just Like This  The Chainsmokers  4752225   
    3127444         2  Something Just Like This  The Chainsmokers  4460815   
    3127644         2  Something Just Like This  The Chainsmokers  3949869   
    3127844         2  Something Just Like This  The Chainsmokers  4374275   
    3128044         2  Something Just Like This  The Chainsmokers  4517368   
    3128244         2  Something Just Like This  The Chainsmokers  4581789   
    3128444         2  Something Just Like This  The Chainsmokers  4649721   
    3128646         4  Something Just Like This  The Chainsmokers  4547924   
    3128846         4  Something Just Like This  The Chainsmokers  4199867   
    3129047         5  Something Just Like This  The Chainsmokers  3733406   
    3129247         5  Something Just Like This  The Chainsmokers  4241126   
    3129447         5  Something Just Like This  The Chainsmokers  4190621   
    3129646         4  Something Just Like This  The Chainsmokers  4297129   
    3129846         4  Something Just Like This  The Chainsmokers  4352601   
    3130046         4  Something Just Like This  The Chainsmokers  4194847   
    3130246         4  Something Just Like This  The Chainsmokers  3749812   
    3130446         4  Something Just Like This  The Chainsmokers  4030370   
    3130646         4  Something Just Like This  The Chainsmokers  4022323   
    3130846         4  Something Just Like This  The Chainsmokers  4037428   
    3131046         4  Something Just Like This  The Chainsmokers  4080722   
    3131246         4  Something Just Like This  The Chainsmokers  4111140   
    3131445         3  Something Just Like This  The Chainsmokers  3989872   
    3131649         7  Something Just Like This  The Chainsmokers  3561983   
    3131848         6  Something Just Like This  The Chainsmokers  3745702   
    3132047         5  Something Just Like This  The Chainsmokers  3830283   
    3132246         4  Something Just Like This  The Chainsmokers  3855992   
    3132446         4  Something Just Like This  The Chainsmokers  3845747   
    3132646         4  Something Just Like This  The Chainsmokers  4011839   
    3132846         4  Something Just Like This  The Chainsmokers  3978468   
    3133046         4  Something Just Like This  The Chainsmokers  3463206   
    ...           ...                       ...               ...      ...   
    3184497        55  Something Just Like This  The Chainsmokers  1091603   
    3184695        53  Something Just Like This  The Chainsmokers  1133446   
    3184894        52  Something Just Like This  The Chainsmokers  1149704   
    3185094        52  Something Just Like This  The Chainsmokers  1139090   
    3185309        67  Something Just Like This  The Chainsmokers  1137304   
    3185512        70  Something Just Like This  The Chainsmokers  1082801   
    3185710        68  Something Just Like This  The Chainsmokers   957821   
    3185905        63  Something Just Like This  The Chainsmokers  1030941   
    3186103        61  Something Just Like This  The Chainsmokers  1083815   
    3186303        61  Something Just Like This  The Chainsmokers  1107312   
    3186506        64  Something Just Like This  The Chainsmokers  1110774   
    3186720        78  Something Just Like This  The Chainsmokers  1105957   
    3186924        82  Something Just Like This  The Chainsmokers  1036470   
    3187162       120  Something Just Like This  The Chainsmokers   949334   
    3187371       129  Something Just Like This  The Chainsmokers   838664   
    3187494        52  Something Just Like This  The Chainsmokers   991213   
    3187691        49  Something Just Like This  The Chainsmokers  1135699   
    3187892        50  Something Just Like This  The Chainsmokers  1205599   
    3188091        49  Something Just Like This  The Chains

In [0]:
################################################################################
# TODO:                                                                        #
# Try to visualize the trend of your favorite song!                            #
################################################################################



################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

### Interactions

Interaction is very useful for exploring datasets. It was time consuming to make charts interactive, thanks to Altair (and the vega-lite behind the scene), it is much easier now. We will go through two examples, but there are more in the [documentation](https://altair-viz.github.io/user_guide/interactions.html), check it out!

In [36]:
# Again, filter, groupby, aggregate, reset_index, apply the tricks we have 
# learnt in this tutorial

df_chart = df_favorite_artist[df_favorite_artist['Continent'] == 'EU'].groupby(['Date', 'Track Name']).sum()['Streams'].reset_index()
df_chart.head()

,Date,Track Name,Streams
0,2017-01-01,Adventure Of A Lifetime,64682
1,2017-01-01,Everglow - Single Version,17622
2,2017-01-01,"Everglow - Single Version, Radio Edit",48726
3,2017-01-01,Hymn For The Weekend,82932
4,2017-01-01,Hymn For The Weekend - Seeb Remix,203571


In [37]:
# Add a scale interaction to allow pan and zoom. Use your scroll wheel to zoom  
# and drag-and-drop to pan. More interaction techniques in documentation

scales = alt.selection_interval(bind='scales')
alt.Chart(df_chart).mark_line().encode(
    x="Date:T",
    y="Streams:Q",
    color="Track Name:N"
).add_selection(
    scales
)

Chart({
  data:             Date                             Track Name  Streams
  0     2017-01-01                Adventure Of A Lifetime    64682
  1     2017-01-01              Everglow - Single Version    17622
  2     2017-01-01  Everglow - Single Version, Radio Edit    48726
  3     2017-01-01                   Hymn For The Weekend    82932
  4     2017-01-01      Hymn For The Weekend - Seeb Remix   203571
  5     2017-01-01                          The Scientist     9786
  6     2017-01-02                Adventure Of A Lifetime    72080
  7     2017-01-02              Everglow - Single Version    25104
  8     2017-01-02  Everglow - Single Version, Radio Edit    61611
  9     2017-01-02                   Hymn For The Weekend    96488
  10    2017-01-02      Hymn For The Weekend - Seeb Remix   228656
  11    2017-01-02                          The Scientist    28389
  12    2017-01-03                Adventure Of A Lifetime    80205
  13    2017-01-03              Everglow - Single Version    26885
  14    2017-01-03  Everglow - Single Version, Radio Edit    71375
  15    2017-01-03                   Hymn For The Weekend    89660
  16    2017-01-03      Hymn For The Weekend - Seeb Remix   230331
  17    2017-01-03                          The Scientist    26971
  18    2017-01-04                Adventure Of A Lifetime    70613
  19    2017-01-04              Everglow - Single Version    29022
  20    2017-01-04  Everglow - Single Version, Radio Edit    82416
  21    2017-01-04                   Hymn For The Weekend    89018
  22    2017-01-04      Hymn For The Weekend - Seeb Remix   234109
  23    2017-01-04                          The Scientist    26658
  24    2017-01-05                Adventure Of A Lifetime    66582
  25    2017-01-05              Everglow - Single Version    32066
  26    2017-01-05  Everglow - Single Version, Radio Edit    66239
  27    2017-01-05                   Hymn For The Weekend    88713
  28    2017-01-05      Hymn For The Weekend - Seeb Remix   240058
  29    2017-01-05                          The Scientist    11008
  ...          ...                                    ...      ...
  1413  2018-01-01      Hymn For The Weekend - Seeb Remix    12748
  1414  2018-01-01                           Viva La Vida    14771
  1415  2018-01-02              Everglow - Single Version    16622
  1416  2018-01-02  Everglow - Single Version, Radio Edit    15058
  1417  2018-01-02                                Fix You    17688
  1418  2018-01-02      Hymn For The Weekend - Seeb Remix    17510
  1419  2018-01-02                          The Scientist     3617
  1420  2018-01-02                           Viva La Vida    17833
  1421  2018-01-03              Everglow - Single Version    22146
  1422  2018-01-03  Everglow - Single Version, Radio Edit    16369
  1423  2018-01-03                          The Scientist     3757
  1424  2018-01-04              Everglow - Single Version    16755
  1425  2018-01-04  Everglow - Single Version, Radio Edit    15945
  1426  2018-01-04      Hymn For The Weekend - Seeb Remix    19684
  1427  2018-01-04                          The Scientist     3586
  1428  2018-01-05              Everglow - Single Version    17607
  1429  2018-01-05  Everglow - Single Version, Radio Edit    13324
  1430  2018-01-05                          The Scientist     3554
  1431  2018-01-06              Everglow - Single Version    16862
  1432  2018-01-06  Everglow - Single Version, Radio Edit    13236
  1433  2018-01-06                          The Scientist     3755
  1434  2018-01-07              Everglow - Single Version    18420
  1435  2018-01-07  Everglow - Single Version, Radio Edit    10930
  1436  2018-01-07                          The Scientist     5981
  1437  2018-01-08              Everglow - Single Version    23065
  1438  2018-01-08  Everglow - Single Version, Radio Edit    15226
  1439  2018-01-08                          The Scientist     5904
  1440  2018-01-09              E

In [38]:
# Interactively focus on data of interest (gray-out other data). We need to make
# a chart, and a selector ponel. The recommended way is to make two charts, 
# putting them side-by-side, just like the juxtaposition example above. Try to 
# click on the squares in the lengend, it acts like a highlighter.

selection = alt.selection_multi(fields=['Track Name'])
color = alt.condition(selection,
                      alt.Color('Track Name:N', legend=None),
                      alt.value('lightgray'))
chart = alt.Chart(df_chart).mark_line().encode(
    x="Date:T",
    y="Streams:Q",
    color=color,
    tooltip=["Track Name", "Streams", "Date"]
)

legend = alt.Chart(df_chart).mark_square().encode(
    y=alt.Y('Track Name:N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection
)

chart | legend

HConcatChart({
  hconcat: [Chart({
    data:             Date                             Track Name  Streams
    0     2017-01-01                Adventure Of A Lifetime    64682
    1     2017-01-01              Everglow - Single Version    17622
    2     2017-01-01  Everglow - Single Version, Radio Edit    48726
    3     2017-01-01                   Hymn For The Weekend    82932
    4     2017-01-01      Hymn For The Weekend - Seeb Remix   203571
    5     2017-01-01                          The Scientist     9786
    6     2017-01-02                Adventure Of A Lifetime    72080
    7     2017-01-02              Everglow - Single Version    25104
    8     2017-01-02  Everglow - Single Version, Radio Edit    61611
    9     2017-01-02                   Hymn For The Weekend    96488
    10    2017-01-02      Hymn For The Weekend - Seeb Remix   228656
    11    2017-01-02                          The Scientist    28389
    12    2017-01-03                Adventure Of A Lifetime    80205
    13    2017-01-03              Everglow - Single Version    26885
    14    2017-01-03  Everglow - Single Version, Radio Edit    71375
    15    2017-01-03                   Hymn For The Weekend    89660
    16    2017-01-03      Hymn For The Weekend - Seeb Remix   230331
    17    2017-01-03                          The Scientist    26971
    18    2017-01-04                Adventure Of A Lifetime    70613
    19    2017-01-04              Everglow - Single Version    29022
    20    2017-01-04  Everglow - Single Version, Radio Edit    82416
    21    2017-01-04                   Hymn For The Weekend    89018
    22    2017-01-04      Hymn For The Weekend - Seeb Remix   234109
    23    2017-01-04                          The Scientist    26658
    24    2017-01-05                Adventure Of A Lifetime    66582
    25    2017-01-05              Everglow - Single Version    32066
    26    2017-01-05  Everglow - Single Version, Radio Edit    66239
    27    2017-01-05                   Hymn For The Weekend    88713
    28    2017-01-05      Hymn For The Weekend - Seeb Remix   240058
    29    2017-01-05                          The Scientist    11008
    ...          ...                                    ...      ...
    1413  2018-01-01      Hymn For The Weekend - Seeb Remix    12748
    1414  2018-01-01                           Viva La Vida    14771
    1415  2018-01-02              Everglow - Single Version    16622
    1416  2018-01-02  Everglow - Single Version, Radio Edit    15058
    1417  2018-01-02                                Fix You    17688
    1418  2018-01-02      Hymn For The Weekend - Seeb Remix    17510
    1419  2018-01-02                          The Scientist     3617
    1420  2018-01-02                           Viva La Vida    17833
    1421  2018-01-03              Everglow - Single Version    22146
    1422  2018-01-03  Everglow - Single Version, Radio Edit    16369
    1423  2018-01-03                          The Scientist     3757
    1424  2018-01-04              Everglow - Single Version    16755
    1425  2018-01-04  Everglow - Single Version, Radio Edit    15945
    1426  2018-01-04      Hymn For The Weekend - Seeb Remix    19684
    1427  2018-01-04                          The Scientist     3586
    1428  2018-01-05              Everglow - Single Version    17607
    1429  2018-01-05  Everglow - Single Version, Radio Edit    13324
    1430  2018-01-05                          The Scientist     3554
    1431  2018-01-06              Everglow - Single Version    16862
    1432  2018-01-06  Everglow - Single Version, Radio Edit    13236
    1433  2018-01-06                          The Scientist     3755
    1434  2018-01-07              Everglow - Single Version    18420
    1435  2018-01-07  Everglow - Single Version, Radio Edit    10930
    1436  2018-01-07                          The Scientist     5981
    1437  2018-01-08              Everglow - Single Version    23065
    1438  2018-01-08  Ever